In [1]:
from utils import AverageMeter, load_inputs, get_png_names, get_ys
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms, utils, models
from model import RecurrentAttention
import torch.nn.functional as F
from config import get_config
from trainer import Trainer
from PIL import Image
from tqdm import tqdm
import pandas as pd
import torchvision
import random
import shutil
import pickle
import torch
import utils
import json
import time

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [2]:
m = open("../../pooling/data/migration_data.json",)
mig_data = json.load(m)
m.close()
mig_data = pd.DataFrame.from_dict(mig_data, orient = 'index').reset_index()
mig_data.columns = ['muni_id', 'num_migrants']
q = 4
mig_data['class'] = pd.qcut(mig_data['num_migrants'], q = q, labels = [i for i in range(q)])
mig_data

,muni_id,num_migrants,class
0,484001001,42055.0,3
1,484001002,4017.0,3
2,484001003,11992.0,3
3,484001004,762.0,2
4,484001005,7551.0,3
...,...,...,...
2326,484032049,2487.0,3
2327,484032050,2024.0,3
2328,484032051,3084.0,3
2329,484032052,2919.0,3


In [3]:
image_names = get_png_names("../../attn/data/MEX/")
y_class, y_mig = get_ys(image_names, mig_data)

In [4]:
c = open("../../archive/CAOE/data/census_data.json",)
census = json.load(c)
c.close()

In [5]:
len(census['484002001'])

202

In [6]:
def get_census(image_names, census):
    census_data = []
    for image in image_names:
        muni_id = image.split("/")[5]
        census_data.append(census[muni_id])
    return census_data
        
census_data = get_census(image_names, census)

In [7]:
train_num = int(25 * .70)
train_indices = random.sample(range(0, 25), train_num)
val_indices = [i for i in range(0, 25) if i not in train_indices]

In [8]:
batch_size = 1

train = [(torchvision.transforms.functional.adjust_brightness(load_inputs(image_names[i]), brightness_factor = 2).squeeze(), y_class[i], y_mig[i], torch.tensor(census_data[i])) for i in train_indices]
val = [(torchvision.transforms.functional.adjust_brightness(load_inputs(image_names[i]), brightness_factor = 2).squeeze(), y_class[i], y_mig[i], torch.tensor(census_data[i])) for i in val_indices]

train_dl = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = True)
val_dl = torch.utils.data.DataLoader(val, batch_size = batch_size, shuffle = True)

In [11]:
config, unparsed = get_config()
trainer = Trainer(config, (train_dl, val_dl))
trainer.train()

0.1s - loss: 63.545 - acc: 0.000 - mae: 62.316:   0%|          | 0/17 [00:00<?, ?it/s]


[*] Train on 17 samples, validate on 8 samples

Epoch: 1/200 - LR: 0.000300


1.0s - loss: 42.166 - acc: 0.000 - mae: 40.894: 100%|██████████| 17/17 [00:01<00:00, 16.41it/s]       


train loss: 2617.248 - train acc: 29.412 - val loss: 5943.591 - val acc: 25.000 - val err: 75.000- train mae: 2615.102 - val mae: 5941.887 [*]


FileNotFoundError: [Errno 2] No such file or directory: './ckpt/ram_4_50x50_0.75_ckpt.pth.tar'